
# prediccion de incidentes viales en la ciudad de Medellin

## importacion librerias necesarias

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers


## lectura y exploracion de los datos

In [2]:
#lectura dataframe saltando las filas malas
df=pd.read_csv("incidentes_viales.csv",on_bad_lines='skip',index_col=False,sep=";",encoding="UTF-8")
print(df.head())

    AÑO  CBML CLASE_ACCIDENTE      DIRECCION       DIRECCION ENCASILLADA  \
0  2015  1013  Caida Ocupante    CR 46 CL 43    CR  046   043  000 00000   
1  2015  1020          Choque  CR 43 A CL 29  CR  043 A   029  000 00000   
2  2015  0205            Otro    CR 46 CL 37    CR  046   037  000 00000   
3  2015  0101       Atropello   CR 34 CL 112    CR  034   112  000 00000   
4  2015  1504          Choque    CR 50 CL 14    CR  050   014  000 00000   

         DISEÑO  EXPEDIENTE      FECHA_ACCIDENTE          FECHA_ACCIDENTES  \
0  Tramo de via  A000259731  21/10/2015 05:58:00  2015-10-21T10:58:00.000Z   
1  Tramo de via  A000261725  05/11/2015 03:00:00  2015-11-05T08:00:00.000Z   
2  Tramo de via  A000259739  21/10/2015 07:40:00  2015-10-21T12:40:00.000Z   
3  Tramo de via  A000259752  21/10/2015 12:00:00  2015-10-21T17:00:00.000Z   
4  Tramo de via  A000259949  03/11/2015 06:16:00  2015-11-03T11:16:00.000Z   

  GRAVEDAD_ACCIDENTE  MES NRO_RADICADO NUMCOMUNA                     BARRI

C:\Users\johan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#revisamos las columnas
print(df.columns)
print("numero de columnas iniciales ",len(df.columns))
print("numero de filas iniciales ",df.shape[0])


Index(['AÑO', 'CBML', 'CLASE_ACCIDENTE', 'DIRECCION', 'DIRECCION ENCASILLADA',
       'DISEÑO', 'EXPEDIENTE', 'FECHA_ACCIDENTE', 'FECHA_ACCIDENTES',
       'GRAVEDAD_ACCIDENTE', 'MES', 'NRO_RADICADO', 'NUMCOMUNA', 'BARRIO',
       'COMUNA', 'LOCATION', 'X', 'Y'],
      dtype='object')
numero de columnas iniciales  18
numero de filas iniciales  270765


## limpieza de datos

In [4]:
#eliminacion de datos con nulos
df.dropna(inplace=True)
#eliminar datos duplicados
df.drop_duplicates(inplace=True)
print(df.shape[0])

250686


In [5]:
#AÑO-Año de ocurrencia del incidente
df["AÑO"]=df["AÑO"].apply(lambda x : re.findall(r"[0-9]{4}",str(x))[0])
print(df["AÑO"].unique())

['2015' '2014' '2016' '2017' '2018' '2019' '2020']


In [6]:
#CBML-Codigo catastral que corresponde al codigo comuna, barrio, manzana, lote catastral de un predio.
print(len(df["CBML"].unique()))

521


In [7]:
#CLASE_ACCIDENTE-Clasificación del IPAT sobre la clase de accidente de transito: choque, atropello, volcamiento,
#caida de ocupante, incendio, u otro (que no corresponde a las anteriores 5 clasificaciones, p. ej: sumersión)
df["CLASE_ACCIDENTE"]=df["CLASE_ACCIDENTE"].apply(lambda x : str(x).replace("Caída de Ocupante","Caida Ocupante").lower())
print((df["CLASE_ACCIDENTE"].unique()))

['caida ocupante' 'choque' 'otro' 'atropello' 'volcamiento' 'incendio']


In [8]:
#DIRECCION-Dirección donde ocurrió el incidente
##aparentemente se encuentra ya formateada
print((df["DIRECCION"].sample(10)))

175886          CQ 1 CL 70
46546        CR 71 CL 89 D
120775         CR 60 CL 50
208510      CR 64 C CL 103
238474         CR 63 CL 56
180934         CR 71 CL 38
242524         CR 62 CL 49
164270         CR 63 CL 50
14557     CL 2 Sur CR 50 E
210956         CR 79 CL 35
Name: DIRECCION, dtype: object


In [9]:
#DIRECCION ENCASILLADA-	Dirección encasillada que entrega el geocodificador
##aparentemente se encuentra ya formateada
##variable con poca informacion util dada la variable anterior--escoger una de las 2
print((df["DIRECCION ENCASILLADA"].sample(10)))

91652       CR  046   046  000 00000
104913    CR  064 C   078  000 00000
199572      CR  062   033  000 00000
15588     DG  074 B   033  000 00000
19282       CL  044   050  000 00000
205650      CL  054   048  000 00000
212437    CL  059 A   039  000 00000
24066     CL  010 A   038  000 00000
103488     CL S 002   052  000 00000
15530       CL  044   098  000 00000
Name: DIRECCION ENCASILLADA, dtype: object


In [10]:
#DISENO-Sitio de la vía donde ocurrió el accidente: Cicloruta, Glorieta, Interseccion, Lote o Predio, Paso a Nivel, Paso Elevado,
#Paso Inferior, Pontón, Puente, Tramo de via, Tunel, Via peatonal
df["DISEÑO"]=df["DISEÑO"].apply(lambda x : str(x).replace("Pont\\xF3n","Pontón").lower())
print((df["DISEÑO"].unique()))

['tramo de via' 'interseccion' 'glorieta' 'lote o predio' 'paso a nivel'
 'puente' 'paso elevado' 'ciclo ruta' 'paso inferior' 'via peatonal'
 'pontón' 'tunel']


In [11]:
#EXPEDIENTE
#Consecutivo que asigna UNE, según el orden de llegada de los expedientes para su diligenciamiento
##variable  aprarentemente con poco valor
print((df["EXPEDIENTE"].sample(10)))

158415    A000840438
137854    A000917715
83977     A000489096
89207     A000586671
129428    A000372657
116460    A000372562
246071    A000669329
66199     A001152234
4244      A000256836
35236     A001003186
Name: EXPEDIENTE, dtype: object


In [12]:
#FECHA_ACCIDENTE
#Fecha del accidente, proviene del IPAT - Informe Policial de accidente de Tránsito
df["FECHA_ACCIDENTE"] = pd.to_datetime(df["FECHA_ACCIDENTE"],dayfirst=True)
print((df["FECHA_ACCIDENTE"].sample(10)))

143519   2016-01-04 12:00:00
54638    2014-10-28 15:40:00
105178   2014-07-22 13:30:00
134529   2018-07-13 16:15:00
254981   2019-06-07 11:15:00
576      2015-11-23 02:20:00
265710   2020-02-05 19:20:00
237805   2016-03-26 10:00:00
63271    2014-07-17 12:20:00
266879   2020-03-02 09:10:00
Name: FECHA_ACCIDENTE, dtype: datetime64[ns]


In [13]:
#FECHA_ACCIDENTES
#Fecha de los accidente (formato YYYY-MM-DD hh:mi:ss), proviene del IPAT - Informe Policial de accidentes de Tránsito
##veriable redundante con la anterior
print((df["FECHA_ACCIDENTES"].head(10)))

0    2015-10-21T10:58:00.000Z
1    2015-11-05T08:00:00.000Z
2    2015-10-21T12:40:00.000Z
3    2015-10-21T17:00:00.000Z
4    2015-11-03T11:16:00.000Z
5    2015-11-05T09:30:00.000Z
6    2015-10-18T09:40:00.000Z
7    2015-11-04T09:46:00.000Z
8    2015-10-21T12:00:00.000Z
9    2015-11-05T12:00:00.000Z
Name: FECHA_ACCIDENTES, dtype: object


In [14]:
#GRAVEDAD_ACCIDENTE
#Clasificación del IPAT - Informe Policial de Accidentes de Tránsito, sobre la gravedad del accidente, corresponde al resultado 
#más grave presentado en el accidente. Daños materiales "Sólo daños", accidente con heridos "Herido", accidente con muertos 
#"Muerto". No indica cantidad
df["GRAVEDAD_ACCIDENTE"]=df["GRAVEDAD_ACCIDENTE"].apply(lambda x : str(x).replace("Solo da\\xF1os","Solo daños").lower())
print((df["GRAVEDAD_ACCIDENTE"].unique()))

['con heridos' 'solo daños' 'con muertos']


In [15]:
#MES
#Mes de ocurrencia del incidente vial
print((df["MES"].unique()))

[10 11 12  9  4  3  5  6  7  8  1  2]


In [16]:
#NRO_RADICADO
#Consecutivo que asigna UNE, según el orden de llegada de los expedientes para su diligenciamiento
#no tiene mucho valor aparentemente para nuestros fines
print((df["NRO_RADICADO"].head()))

0    1508668
1    1510621
2    1508691
3    1508694
4    1510634
Name: NRO_RADICADO, dtype: object


In [17]:
#NUMCOMUNA
#Numero de la comununa en la que ocurrio incidente vial
#se eliminan las filas con valores que no corespondan con 
df=df[df["NUMCOMUNA"]!="AU"]
df=df[df["NUMCOMUNA"]!="Sin Inf"]
df=df[df["NUMCOMUNA"]!="In"]
df=df[df["NUMCOMUNA"]!="SN"]
df=df[df["NUMCOMUNA"]!="0"]
#asuminos que las comunas 60,70,80,90 son las comunas 6,7,8,9 respectivamente
df["NUMCOMUNA"]=df["NUMCOMUNA"].apply(lambda x : (int(x)))
df["NUMCOMUNA"]=df["NUMCOMUNA"].apply(lambda x : x/10 if x >20 else x)
df["NUMCOMUNA"]=df["NUMCOMUNA"].apply(lambda x : str(int(x)))
print(sorted(df["NUMCOMUNA"].unique()))

['1', '10', '11', '12', '13', '14', '15', '16', '2', '3', '4', '5', '6', '7', '8', '9']


In [18]:
#BARRIO
print(len(df["BARRIO"].unique()))
#Barrio de ocurrencia del incidente vial
#ó por o
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).lower().replace("\\xf3","o").replace("ó","o"))
#é por e 
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xe9","e").replace("é","e"))
#á por a 
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xc1","a").replace("\\xe1","a").replace("á","a"))
#ñ
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xf1","ñ"))
#\\xed í por i
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xed","i").replace("í","i"))
#\\xfa ú por u
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xfa","u").replace("ú","u"))
print((df["BARRIO"].unique()))

488
['barrio colon' 'san diego' 'la francia' 'santo domingo savio no. 1'
 'santa fe' 'la florida' 'los conquistadores' 'la alpujarra' 'villa nueva'
 'facultad de minas u. nacional' 'la esperanza' 'perpetuo socorro'
 'campo valdes no. 1' 'miraflores' 'oleoducto' 'terminal de transporte'
 'gerona' 'u.d. atanasio girardot' 'alejandro echavarria'
 'cabecera san antonio de prado' 'manila' 'los balsos no.2' 'moravia'
 'tenche' 'universidad nacional' 'los colores' 'caribe'
 'juan xxiii la quiebra' 'belalcazar' 'santa cruz' 'sucre' 'la candelaria'
 'el estadio' 'suramericana' 'la gloria' 'centro administrativo'
 'las estancias' 'aures no. 2' 'los angeles' 'castilla' 'la salle'
 'la america' 'picacho' 'bosques de san pablo' 'la aguacatala'
 'playon de los comuneros' 'estacion villa' 'la colina' 'popular'
 'villa del socorro' 'cucaracho' 'santa maria de los angeles'
 'el chagualo' 'la mansion' 'san benito' 'la castellana' 'jesus nazareno'
 'loreto' 'asomadera no. 1' 'el salvador' 'cristo rey' 'm

In [19]:
#COMUNA
#Denominación con la cual se identifica cada Comuna o Corregimiento. 01:Popular 02:Santa Cruz 03:Manrique 04:Aranjuez 05:Castilla 06:Doce de Octubre 07:Robledo 08:Villa Hermosa 09:Buenos Aires 10:La Candelaria 11:Laureles - Estadio 12:La América 13:San Javier 14:El Poblado 15:Guayabal 16:Belén 50:San Sebastián de Palmitas 60:San Cristobal 70:Altavista 80:San Antonio de Prado 90:Santa Elena 99:Toda la Ciudad
##variable redundante
#ó por o
df["COMUNA"]=df["COMUNA"].apply(lambda x : str(x).lower().replace("\\xf3","o").replace("ó","o"))
#á por a 
df["COMUNA"]=df["COMUNA"].apply(lambda x : str(x).replace("\\xc1","a").replace("\\xe1","a").replace("á","a"))
#é por e 
df["COMUNA"]=df["COMUNA"].apply(lambda x : str(x).replace("\\xe9","e").replace("é","e"))
print((df["COMUNA"].unique()))

['la candelaria' 'santa cruz' 'popular' 'guayabal' 'el poblado'
 'laureles estadio' 'robledo' 'doce de octubre' 'aranjuez' 'buenos aires'
 'castilla' 'corregimiento de san antonio de prado' 'san javier'
 'villa hermosa' 'belen' 'manrique' 'la america'
 'corregimiento de altavista' 'corregimiento de san cristobal'
 'corregimiento de santa elena' 'in'
 'corregimiento de san sebastian de palmitas' 'au']


In [20]:
#LOCATION
#Fuente de información con la cual se realizó la geocodificación
print((df["LOCATION"].head(10)))

0    [-75.5688011014, 6.24312304123]
1    [-75.5693883283, 6.22870030622]
2    [-75.5503911403, 6.29869502502]
3    [-75.5442195069, 6.30019383306]
4     [-75.578180998, 6.21699243701]
5      [-75.561118009, 6.2047775959]
6    [-75.5810144962, 6.24576066323]
7    [-75.5752597216, 6.24685341912]
8    [-75.5624178534, 6.25341454089]
9    [-75.5931430393, 6.27330476643]
Name: LOCATION, dtype: object


In [21]:
#X
#Coordenada X en metros del accidente, en sistema de coordenadas MAGNA Medellin Local
print(df["X"].head(10))

0    834949.69
1    834880.17
2    837004.94
3    837688.53
4    833903.11
5    835788.30
6    833598.49
7    834235.89
8    835659.53
9    832264.64
Name: X, dtype: float64


In [22]:
#Y
#Coordenada Y en metros del accidente, en sistema de coordenadas MAGNA Medellin Local
print(df["Y"].head(10))

0    1182357.34
1    1180762.02
2    1188499.17
3    1188663.05
4    1179469.60
5    1178113.02
6    1182652.97
7    1182772.04
8    1183493.83
9    1185703.90
Name: Y, dtype: float64


In [23]:
#Eliminar duplicados
df.drop_duplicates(inplace=True)
#borrar columnas con datos redudantes o poco aportantes
df.drop(labels=["DIRECCION ENCASILLADA","EXPEDIENTE","FECHA_ACCIDENTES","NRO_RADICADO","COMUNA"],axis=1,inplace=True)

In [24]:
df.to_csv("datosLimpios.csv")
#guardamos los datos en una nueva varible
data=df.copy()
df

,AÑO,CBML,CLASE_ACCIDENTE,DIRECCION,DISEÑO,FECHA_ACCIDENTE,GRAVEDAD_ACCIDENTE,MES,NUMCOMUNA,BARRIO,LOCATION,X,Y
0,2015,1013,caida ocupante,CR 46 CL 43,tramo de via,2015-10-21 05:58:00,con heridos,10,10,barrio colon,"[-75.5688011014, 6.24312304123]",834949.69,1182357.34
1,2015,1020,choque,CR 43 A CL 29,tramo de via,2015-11-05 03:00:00,solo daños,11,10,san diego,"[-75.5693883283, 6.22870030622]",834880.17,1180762.02
2,2015,0205,otro,CR 46 CL 37,tramo de via,2015-10-21 07:40:00,con heridos,10,2,la francia,"[-75.5503911403, 6.29869502502]",837004.94,1188499.17
3,2015,0101,atropello,CR 34 CL 112,tramo de via,2015-10-21 12:00:00,con heridos,10,1,santo domingo savio no. 1,"[-75.5442195069, 6.30019383306]",837688.53,1188663.05
4,2015,1504,choque,CR 50 CL 14,tramo de via,2015-11-03 06:16:00,solo daños,11,15,santa fe,"[-75.578180998, 6.21699243701]",833903.11,1179469.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
270760,2020,1004,caida ocupante,CR 62 CL 59,tramo de via,2020-08-31 18:50:00,con heridos,8,10,el chagualo,"[-75.5731343044, 6.26063534822]",834475.51,1184295.99
270761,2020,1115,otro,CL 50 CR 74,tramo de via,2020-08-31 18:50:00,con heridos,8,11,cuarta brigada,"[-75.5902062951, 6.26040455473]",832585.61,1184275.87
270762,2020,515,choque,CL 80 CR 65,tramo de via,2020-08-31 19:05:00,solo daños,8,5,cementerio universal,"[-75.5735398111, 6.27625838886]",834435.55,1186024.41
270763,2020,207,atropello,CL 104 CR 45,tramo de via,2020-08-31 19:30:00,con heridos,8,2,villa del socorro,"[-75.5515085985, 6.29359892373]",836879.66,1187935.77


In [25]:
#df=pd.read_csv("datosLimpios.csv")

## 1 - Entrenamiento de un modelo predictivo
Se deberá construir un modelo que permita predecir la accidentalidad por tipo de accidente a nivel semanal, mensual y diario. Para esto se deberán considerar fechas especiales.

In [26]:
df=data
df["DIA"]=df["FECHA_ACCIDENTE"].apply(lambda x: x.date())
df=df.groupby(["DIA","CLASE_ACCIDENTE","NUMCOMUNA"]).count()
df.reset_index(inplace=True)
df["CANTIDAD"]=df["AÑO"]
df=df[["DIA","CLASE_ACCIDENTE","CANTIDAD","NUMCOMUNA"]]
df

,DIA,CLASE_ACCIDENTE,CANTIDAD,NUMCOMUNA
0,2014-07-04,choque,8,10
1,2014-07-04,choque,4,11
2,2014-07-04,choque,1,12
3,2014-07-04,choque,2,14
4,2014-07-04,choque,2,15
...,...,...,...,...
85313,2020-08-31,volcamiento,2,14
85314,2020-08-31,volcamiento,1,15
85315,2020-08-31,volcamiento,1,16
85316,2020-08-31,volcamiento,1,2


In [27]:
import holidays_co as co
from datetime import timedelta
#dia_festivo 1 si 0 no
df["dia_festivo".upper()]=df["DIA"].apply(lambda x : str(co.is_holiday_date(x)))
#sigiente_dia_festivo
df["sigiente_dia_festivo".upper()]=df["DIA"].apply(lambda x : str(co.is_holiday_date(x+timedelta(days=1))))
#anterior_dia_festivo
df["anterior_dia_festivo".upper()]=df["DIA"].apply(lambda x : str(co.is_holiday_date(x-timedelta(days=1))))
df["AÑO"]=df["DIA"].apply(lambda x : pd.Timestamp(x).year)
df["MES"]=df["DIA"].apply(lambda x : pd.Timestamp(x).month)
df["dia_de_la_semana".upper()]=df["dia".upper()].apply(lambda x : pd.Timestamp(x).day_of_week)
df["semana_del_año".upper()]=df["dia".upper()].apply(lambda x : pd.Timestamp(x).week)
df["dia_del_mes".upper()]=df["dia".upper()].apply(lambda x : pd.Timestamp(x).day)
df.tail(10)
dfaux=df.copy()


Sea X el con junto de parametros 

### Entrenamiento y validación
Los modelos predictivos se deberán construir con los datos de los años 2014, 2015, 2016, 2018. Se usan los accidentes del año 2019 y 2020 (¡alerta!) para validar los modelos.
Se deberá analizar el comportamiento del modelo predictivo en los años 2021 y 2022 y explicar las posibles desviaciones.

In [28]:
dfaux=df.copy()
#dfaux=pd.concat([dfaux,pd.get_dummies(dfaux.pop("NUMCOMUNA"),drop_first=True)], axis=1)
train=dfaux[(dfaux["AÑO"]) <2019].drop(["DIA"],axis=1)
test=dfaux.drop(train.index).drop(["DIA"],axis=1)
X_train=train.copy()
Y_train=X_train.pop("CANTIDAD")
X_test=test.copy()
Y_test=X_test.pop("CANTIDAD")
print(sorted(X_train.NUMCOMUNA.unique()))
print(sorted(X_test.NUMCOMUNA.unique()))

['1', '10', '11', '12', '13', '14', '15', '16', '2', '3', '4', '5', '6', '7', '8', '9']
['1', '10', '11', '12', '13', '14', '15', '16', '2', '3', '4', '5', '6', '7', '8', '9']


,DIA,CLASE_ACCIDENTE,CANTIDAD,NUMCOMUNA,DIA_FESTIVO,SIGIENTE_DIA_FESTIVO,ANTERIOR_DIA_FESTIVO,AÑO,MES,DIA_DE_LA_SEMANA,SEMANA_DEL_AÑO,DIA_DEL_MES
0,2014-07-04,choque,8,10,False,False,False,2014,7,4,27,4
1,2014-07-04,choque,4,11,False,False,False,2014,7,4,27,4
2,2014-07-04,choque,1,12,False,False,False,2014,7,4,27,4
3,2014-07-04,choque,2,14,False,False,False,2014,7,4,27,4
4,2014-07-04,choque,2,15,False,False,False,2014,7,4,27,4


In [29]:
from sklearn.compose import make_column_selector
    # Importe make_column_transformer
from sklearn.compose import make_column_transformer
    # Importe SelectKBest
from sklearn.feature_selection import SelectKBest
    # Importe f_regression
from sklearn.feature_selection import f_regression
    # Importe LinearRegression
from sklearn.linear_model import LinearRegression
    # Importe GridSearchCV
from sklearn.model_selection import GridSearchCV
    # Importe Pipeline
from sklearn.pipeline import Pipeline
    # Importe OneHotEncoder
from sklearn.preprocessing import OneHotEncoder


pipeline = Pipeline(
        steps=[
            # Paso 1: Construya un column_transformer que aplica OneHotEncoder a las
            # variables categóricas, y no aplica ninguna transformación al resto de
            # las variables.
            (
                "column_transfomer",
                make_column_transformer(
                    (
                        OneHotEncoder(),
                        make_column_selector(dtype_include=object),
                    ),
                    remainder='passthrough',
                ),
            ),
            # Paso 2: Construya un selector de características que seleccione las K
            # características más importantes. Utilice la función f_regression.
            (
                "selectKBest",
                SelectKBest(score_func=f_regression),
            ),
            # Paso 3: Construya un modelo de regresión lineal.
            (
                "linearRegression",
                LinearRegression(),
            ),
        ],
    )

In [30]:
 param_grid = {
        "selectKBest__k": (1, 33),
    }

In [31]:
gridSearchCV = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=5,
        scoring="neg_mean_squared_error",
        refit=True,
        return_train_score=True,
    )

In [32]:
gridSearchCV.fit(X_train, Y_train.values.ravel())

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('column_transfomer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001927F7452C8>)])),
                                       ('selectKBest',
                                        SelectKBest(score_func=<function f_regression at 0x00000192019230D8>)),
                                       ('linearRegression',
                                        LinearRegression())]),
             param_grid={'selectKBest__k': (1, 33)}, return_train_score=True,
             scoring='neg_mean_squared_error')

In [33]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# Evalúe el modelo con los conjuntos de entrenamiento y prueba.
Y_train_pred = gridSearchCV.predict(X_train)
Y_test_pred = gridSearchCV.predict(X_test)

    # Compute el error cuadratico medio de entrenamiento y prueba. Redondee los
    # valores a dos decimales.

mse_train = r2_score(
        Y_train,
        Y_train_pred,
    ).round(2)

mse_test = r2_score(
        Y_test,
        Y_test_pred,
    ).round(2)

In [34]:
print(mse_test,mse_train)

0.38 0.45


In [35]:
gridSearchCV.best_params_

{'selectKBest__k': 33}

In [36]:
from sklearn.metrics import mean_absolute_percentage_error,median_absolute_error
mape_train = median_absolute_error(
        Y_train,
        Y_train_pred,
    ).round(2)

mape_test = median_absolute_error(
        Y_test,
        Y_test_pred,
    ).round(2)

In [37]:
print(mape_train,mape_test)

1.38 1.43


In [38]:
resultTest=pd.DataFrame()
resultTest["esperado"]=Y_test
resultTest["predicho"]=Y_test_pred

In [39]:
resultTest.head()

,esperado,predicho
63232,1,-1.151184
63233,1,-0.982361
63234,1,0.804199
63235,1,0.492371
63236,1,-1.510193
